【東吳大學資管系】資料工程實務與應用<br>
【授課老師】[陳祥輝 (Email : HsiangHui.Chen@gmail.com)](mailto:HsiangHui.Chen@gmail.com)<br>
【facebook】[陳祥輝老師的臉書 (歡迎加好友)](https://goo.gl/osivhx)<br>
【相關課程】[陳祥輝老師的課程]
- [【東吳大學推廣部】](https://www.ext.scu.edu.tw/courses_search.php?key=陳祥輝)
- [【臺北大學推廣教育】](https://dce.ntpu.edu.tw/page-search.php?q=陳祥輝)

In [ ]:
!conda env list

#### <font color=#FF0000>【安裝相關模組】</font>
- !pip install --upgrade pymssql
- !pip install --upgrade pyodbc

In [1]:
!pip install -U pyodbc

  Attempting uninstall: pyodbc
    Found existing installation: pyodbc 4.0.32
    Uninstalling pyodbc-4.0.32:
      Successfully uninstalled pyodbc-4.0.32


In [2]:
!pip install -U pymssql

In [1]:
# coding=utf-8
from platform import python_version
import os, time, socket, glob, pickle, joblib
import pandas as pd
import pyodbc, pymssql

import warnings
warnings.filterwarnings(action="ignore")


print("【現在時間】{}".format(time.strftime("%Y/%m/%d %H:%M:%S")))
print("【所在目錄】{}".format(os.getcwd()))
print("【主機名稱】{} ({})".format(socket.gethostname(),socket.gethostbyname(socket.gethostname())))
print("【Python】{}".format(python_version()))
print("【pymssql】{}".format(pymssql.__version__))
print("【pyodbc】{}".format(pyodbc.version))

%autosave 120

【現在時間】2022/11/23 13:32:10
【所在目錄】C:\資料工程
【主機名稱】LAPTOP-HN8S2UQE (163.14.34.76)
【Python】3.9.12
【pymssql】2.2.7
【pyodbc】4.0.35


Autosaving every 120 seconds


#### 【DB 連線的步驟】
1. 建立連線(通道)
2. 透過此連線(通道)進行存、取資料
3. 關閉連線(通道)

- <img src='http://180.218.162.136:2018/Images/Python/db_connection.jpg' width=400 align=left></img>

【DB伺服器資訊】
- MS SQL SERVER
    - 【伺服器】163.14.73.23
    - 【PORT】1433
    - 【Database】PyDB
    - 【user】student
    - 【password】student1111
    
【SQL語法】
<pre>
select a.County, a.SiteName, a.NO, a.NO2
from AirQty as a
where a.County in ('臺北市', '新北市', '高雄市')
</pre>

### <font color='red'>使用 pymssql 模組連線至 MS SQL SERVER</font>
- 安裝模組與匯入
    - pip install --upgrade pymssql
    - import pymssql
- pymssql.connect()
    - server : 伺服器的位址
    - port : 埠號
    - database : 資料庫名稱
    - user : 使用者帳號
    - password : 使用者密碼    

- TCP : 
    - 可靠傳輸，一個bit都不能loss掉
    - 同步傳輸，沒有回應就會重送
    - 效率較差
- UDP :
    - 不可靠傳輸
    - 非同步傳輸，能丟多快就多快，漏接就算了
    - 效率較高

#### Step 1 : 建立連線(通道)

In [2]:
conn = pymssql.connect(server = '163.14.73.23',
                       port = '1433',              # 每個資料庫都會有固定申請的port號
                       database = 'PyDB',
                       user = 'student',
                       password = 'student1111')

#### Step 2 : 執行 SQL / T-SQL 語法

In [3]:
sqlStr = """
select a.County, a.SiteName, a.NO, a.NO2
from AirQty as a
where a.County in ('臺北市', '新北市', '高雄市')
"""

airQty = pd.read_sql(sql=sqlStr, con=conn)
airQty.head()

,County,SiteName,NO,NO2
0,高雄市,復興,1.08,5.1
1,新北市,永和,2.68,9.3
2,新北市,三重,45.20,25.0
3,臺北市,陽明,0.60,7.9
4,高雄市,小港,1.85,9.0


In [4]:
sqlStr = """
select * from iris
"""

iris = pd.read_sql(sql=sqlStr, con=conn)
iris.head()

,花萼長度,花萼寬度,花瓣長度,花瓣寬度,種類
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


#### Step 3 : 結束連線

In [5]:
conn.close()

### <font color='red'>使用 pyodbc 模組連線至 MS SQL SERVER</font>
- 安裝模組與匯入
    - pip install --upgrade pyodbc
    - import pyodbc
- pyodbc.connect()的連線字串
    - 'DSN=yourDSN;UID=user;PWD=password'
    - 'DRIVER={SQL Server Native Client 11.0};SERVER=localhost;DATABASE=testdb;UID=user;PWD=password'
    - dsn : Data Source Name

#### <font color='blue'>【透過ODBC的DSN(Data Source Name)連線】</font>
- DSN : Data Source Name (資料來源名稱)

#### Step 1 : 建立連線(通道)

In [6]:
conn = pyodbc.connect('DSN=dsnPyDB;UID=student;PWD=student1111')

#### Step 2 : 執行 SQL / T-SQL 語法

In [7]:
sqlStr = """
select a.County, a.SiteName, a.NO, a.NO2
from AirQty as a
where a.County in ('臺北市', '新北市', '高雄市')
"""

airQty = pd.read_sql(sql=sqlStr, con=conn)
airQty.head()

,County,SiteName,NO,NO2
0,高雄市,復興,1.08,5.1
1,新北市,永和,2.68,9.3
2,新北市,三重,45.20,25.0
3,臺北市,陽明,0.60,7.9
4,高雄市,小港,1.85,9.0


#### Step 3 : 結束連線

In [5]:
conn.close()

#### <font color='blue'>【查看本機有哪些ODBC的驅動程式】</font>

In [9]:
pyodbc.drivers()

['SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'SQL Server Native Client 11.0',
 'ODBC Driver 17 for SQL Server']

#### <font color='blue'>【透過ODBC的DRIVER(驅動程式)直接連線】</font>

#### Step 1 : 建立連線(通道)

In [10]:
conn = pyodbc.connect("DRIVER={SQL Server Native Client 11.0};SERVER=163.14.73.23;"
                      "DATABASE=PyDB;UID=student;PWD=student1111")

#### Step 2 : 執行 SQL / T-SQL 語法

In [11]:
sqlStr = """
select a.County, a.SiteName, a.NO, a.NO2
from AirQty as a
where a.County in ('臺北市', '新北市', '高雄市')
"""

airQty = pd.read_sql(sql=sqlStr, con=conn)
airQty.head()

,County,SiteName,NO,NO2
0,高雄市,復興,1.08,5.1
1,新北市,永和,2.68,9.3
2,新北市,三重,45.20,25.0
3,臺北市,陽明,0.60,7.9
4,高雄市,小港,1.85,9.0


#### Step 3 : 結束連線

In [12]:
conn.close()

### <font color='red'>使用 pyodbc 模組連線至 MS ACCESS</font>

#### Step 1 : 建立連線(通道)

In [13]:
conn = pyodbc.connect("DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
                      "DBQ=C:\Data\PyData\PyData.accdb")

#### Step 2 : 執行 SQL / T-SQL 語法

In [14]:
sqlStr = """
select a.County, a.SiteName, a.NO, a.NO2
from AirQty as a
where a.County in ('臺北市', '新北市', '高雄市')
"""

airQty = pd.read_sql(sql=sqlStr, con=conn)
airQty.head()

,County,SiteName,NO,NO2
0,高雄市,復興,1.08,5.1
1,新北市,永和,2.68,9.3
2,新北市,三重,45.20,25.0
3,臺北市,陽明,0.60,7.9
4,高雄市,小港,1.85,9.0


#### Step 3 : 結束連線

In [15]:
conn.close()

### <font color='red'>MS SQL SERVER 的資料讀、寫，以及 T-SQL 的執行</font>

<pre>
create table tdale
(
sid char(10) primary key,
sname varchar(100),
birthday date
)

insert into tdale (sid, sname, birthday)
values ('1', 'dale', '2020/01/01')

select * from tdale

update tdale
set sname = 'Dale Chen'
  , birthday = '2020/02/02'
where sid = '1'

delete tdale
where sid = '1'

drop table tdale

exec sp_who2
</pre>

#### Step 1 : 建立連線(通道)

In [20]:
conn = pyodbc.connect("DRIVER={SQL Server Native Client 11.0};SERVER=163.14.73.23;"
                      "DATABASE=PyDB;UID=student;PWD=student1111")

#### Step 2 : 執行 SQL / T-SQL 語法

In [21]:
conn.autocommit = True
cursor = conn.cursor()

sqlStr = """
    create table t09156142
    (
    sid char(10) primary key,
    sname varchar(100),
    birthday date
    )
"""

cursor.execute(sqlStr)

In [26]:
sqlStr = """
    insert into t09156142 (sid, sname, birthday)
    values ('1', 'dale', '2020/01/01')
"""

try :
    cursor.execute(sqlStr)
except :
    pass # 什麼都不做

# conn.commit() 如果原本設為False，要寫這行，不然資料不會寫入

#### Step 3 : 結束連線

In [12]:
conn.close()